# Тема: Изучение базы данных книжного сервиса

**Описание данных**

Таблица books

Содержит данные о книгах:

* `book_id` — идентификатор книги;

* `author_id` — идентификатор автора;

* `title` — название книги;

* `num_pages` — количество страниц;

* `publication_date` — дата публикации книги;

* `publisher_id` — идентификатор издателя.

*Таблица authors*

Содержит данные об авторах:

* `author_id` — идентификатор автора;

* `author` — имя автора.

*Таблица publishers*

Содержит данные об издательствах:

* `publisher_id` — идентификатор издательства;

* `publisher` — название издательства;

*Таблица ratings*

Содержит данные о пользовательских оценках книг:

* `rating_id` — идентификатор оценки;

* `book_id` — идентификатор книги;

* `username` — имя пользователя, оставившего оценку;

* `rating` — оценка книги.

*Таблица reviews*

Содержит данные о пользовательских обзорах:

* `review_id` — идентификатор обзора;

* `book_id` — идентификатор книги;

* `username` — имя автора обзора;

* `text` — текст обзора.


**Цель проекта**

Проанализировать базу данных сервиса продаж книг.

**Задачи проекта**

Для достижения цели проекта выполним следуюзие задачи:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — исключим брошюры;
- Определим автора с самой высокой средней оценкой книг — в учет пойдут книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Схема данных**

<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png" width=900 height=700 />

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследуем таблицы

In [2]:
def select(table):
    return pd.io.sql.read_sql(table, con = engine) #функция для вывода запроса

In [3]:
# составим запросы на таблицы
books = ''' 
SELECT *
FROM books
LIMIT (5)
'''

authors = '''
SELECT *
FROM authors
LIMIT (5)
'''

publishers = '''
SELECT *
FROM publishers
LIMIT (5)
'''

ratings = '''
SELECT *
FROM ratings
LIMIT (5)
'''

reviews = '''
SELECT *
FROM reviews
LIMIT (5)
'''

In [4]:
#цикл для публикации таблиц
tables = [books, authors, publishers, ratings, reviews]
for i in tables:
    display(select(i))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года;

In [5]:
millenium_books = '''

SELECT COUNT(DISTINCT book_id) 

FROM books

WHERE publication_date >= '2000-01-01'                 /* установим дату включая 1 января*/

'''

select(millenium_books)

,count
0,821


После 1 января 2000 года вышла 821 книга.

## Для каждой книги посчитаем количество обзоров и среднюю оценку;

Для того, чтобы посчитать количество обзоров и среднюю оценку нам понадобится еще 2 таблицы: rewiews и rating. Присоединим их и выведем нужный запрос.

In [6]:
rating_by_books = '''
SELECT 
    title,
    COUNT(DISTINCT review_id) as number_of_review ,             /* посчитаем количество оригинальных отзывов*/
    AVG(rating) as mean                                         /* посчитаем среднюю оценку*/

FROM books as b 

LEFT JOIN ratings as r ON b.book_id = r.book_id                 /* присоедними таблицы*/
LEFT JOIN reviews as rew ON b.book_id = rew.book_id

GROUP BY title

ORDER BY number_of_review DESC                                  /* вывдим в порядке убывания по количеству отзывов*/



'''

select(rating_by_books)

,title,number_of_review,mean
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


Максимальное количество обзоров на книгу Memoirs of a Geisha.

In [7]:
count_title = '''

SELECT COUNT(title)

FROM books

'''
select(count_title)

,count
0,1000


Всего в таблице books 1000 названий книг, а в выводимой нами таблице 999. Так как мы группировали по title, то мы сгруппировали два повторяющихся названия. Найду это дублирующее название.

In [8]:
duplicates = '''
WITH data as (
   SELECT *,
   ROW_NUMBER() OVER (PARTITION BY title) as rowNumber 
   FROM books
)
SELECT
  * 
FROM data 
WHERE
 rowNumber > 1
 
 '''
select(duplicates)

,book_id,author_id,title,num_pages,publication_date,publisher_id,rownumber
0,427,39,Memoirs of a Geisha,503,2005-11-22,311,2


У нас в таблице books дублируется title - Memoirs of a Geisha. оэтому чтобы не групировать дуюликаты названий, сгруппируем по book_id.

In [9]:
rating_by_books = '''
SELECT 
    b.book_id,
    title,
    COUNT(DISTINCT review_id) as number_of_review ,             /* посчитаем количество оригинальных отзывов*/
    AVG(rating) as mean                                         /* посчитаем среднюю оценку*/

FROM books as b 

LEFT JOIN ratings as r ON b.book_id = r.book_id                 /* присоедними таблицы*/
LEFT JOIN reviews as rew ON b.book_id = rew.book_id

GROUP BY b.book_id

ORDER BY number_of_review DESC                                  /* вывдим в порядке убывания по количеству отзывов*/



'''

select(rating_by_books)

,book_id,title,number_of_review,mean
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Итого, максимальное количество обзоров на книгу Twilight (Twilight #1)

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;

Для определения издательства выпустившего наибольшее количество книг более 50 страниц, присоединим к таблице books таблицу publishers.

In [10]:
thick_publisher = '''

SELECT  
    publisher,
    COUNT(publisher)
        
    
FROM books as b 

LEFT JOIN publishers as p ON b.publisher_id = p.publisher_id      /* присоединим таблицу publishers*/           

WHERE num_pages > 50                                              /* установим условие "более 50 страниц"*/  

GROUP BY publisher

ORDER BY count DESC                                               /* выведем количество в порядке убывания*/ 

LIMIT (1)

'''

select(thick_publisher)

,publisher,count
0,Penguin Books,42


Наибольшее количество книг более 50 страниц напечатало издательство Penguin Books - 42 книги. 

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

Для решения этой задачи нам понадобится присоединить к таблице books еще 2 таблицы: ratings и authors.

In [11]:
name_authors = '''

SELECT 
    author,
    AVG(rating) as rating                               /* считаем средний рейтинг*/
    
FROM books as b 
            LEFT JOIN(                                   /* сделаем подзапрос из таблицы ratings и присоединим*/
                        SELECT 
                            book_id,
                            AVG(rating) as rating,                   /*считаем средний рейтинг*/ 
                            COUNT(DISTINCT rating_id) as rating_id   /*считаем количество оценок*/
                        FROM ratings
                        GROUP BY book_id                    
                        
                        ) as r ON b.book_id = r.book_id

          
LEFT JOIN authors as a ON b.author_id = a.author_id       /* присоединим таблицу authors*/ 

WHERE rating_id >= 50                                     /* условие с 50 и более оценками*/


GROUP BY author                                           /* группируем по автору*/



ORDER BY rating DESC                                     /* выведем рейтинг в порядке убывания*/

LIMIT (1)

'''

select(name_authors)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка среди книг с 50 и более оценками принадлежит автору J.K. Rowling/Mary GrandPré

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Для данной цели воспользуемся временными таблицами:

In [12]:
mean_reviews_by_user = '''

WITH                                                      /*создадим временную таблицу в которой будут храниться пользователи,
                                                         который поставили больше 50 оценок*/ 
fifty AS ( 

       SELECT                                           
           username,
           COUNT(DISTINCT rating_id)

       FROM ratings
        
       GROUP BY username
        
       HAVING COUNT(DISTINCT rating_id) > 50 
),
                                                          /*создадим временную таблицу в которой будет храниться информация о
                                                          количестве сделаных обзоров по пользователям*/ 
number_of_reviews AS(

        SELECT                                                 
            username,
            COUNT(DISTINCT review_id) as mean_review
            
        FROM reviews
        
        GROUP BY username
       

)


SELECT AVG(mean_review) AS mean_review                      /*посчитаем среднее по основному запросу*/

FROM fifty LEFT JOIN number_of_reviews ON  fifty.username=number_of_reviews.username  

                                                           /*соединям две временные таблицы*/

'''

select(mean_reviews_by_user).head()

,mean_review
0,24.333333


Итак, среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24.33. 

## Заключение

После изучения базы данных пришли к следующим заключениям:

1) После 1 января 2000 года вышла 821 книга.

2) Итого, максимальное количество обзоров на книгу Twilight (Twilight #1)

3) Penguin Books выпустила 42 книги толще 50 страниц.

4) Самая высокая средняя оценка среди книг с 50 и более оценками принадлежит автору J.K. Rowling/Mary GrandPré

5) Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24.33.

6) В БД присутствует 2 книги c названием Memoirs of a Geisha